In [1]:
import aes2_added_fb_prize_as_features_preprocessing

CUDA_AVAILABLE = True


In [2]:
import gc

gc.collect()

118

In [3]:
import pickle

with open("/home/mcq/GitHub/aes2/train_data/train_feats.pickle", "rb") as f:
    train_feats = pickle.load(f)
with open("/home/mcq/GitHub/aes2/train_data/X.pickle", "rb") as f:
    X = pickle.load(f)
with open("/home/mcq/GitHub/aes2/train_data/y.pickle", "rb") as f:
    y = pickle.load(f)
with open("/home/mcq/GitHub/aes2/train_data/y_split.pickle", "rb") as f:
    y_split = pickle.load(f)
with open(
    "/home/mcq/GitHub/aes2/train_data/feature_select.pickle", "rb"
) as f:
    feature_select = pickle.load(f)
    
aes2_added_fb_prize_as_features_preprocessing.feature_select = feature_select

In [4]:
train_feats.iloc[:5, -8:]

,cohesion,syntax,vocabulary,phraseology,grammar,conventions,argument_0,argument_1
0,2.838330,2.648617,3.225708,2.852826,2.682580,2.372594,0.184659,0.815341
1,3.301917,3.215416,3.436492,3.428899,3.493132,2.969440,0.869833,0.130167
2,4.223053,4.069751,4.395761,4.302904,4.315377,4.023932,0.124480,0.875520
3,3.695875,3.485972,4.000536,3.655286,3.484632,3.629644,0.675310,0.324689
4,3.449726,3.228402,3.503447,3.219933,3.524052,3.082691,0.935159,0.064841


In [5]:
X.shape

(17307, 13000)

In [6]:
import numpy as np

def clean_feature_names(features):
    illegal_chars = ['[', ']', '<', '>']
    cleaned_features = []
    for feature in features:
        for char in illegal_chars:
            feature = feature.replace(char, 'lessthan')
        cleaned_features.append(feature)
    return cleaned_features

train_feats.columns = clean_feature_names(train_feats.columns)
feature_select = clean_feature_names(feature_select)
X = train_feats[feature_select].astype(np.float32).values

# Find Threshold

In [ ]:
def find_thresholds(true, pred, steps=50):

    # SAVE TRIALS FOR PLOTTING
    xs = [[],[],[],[],[]]
    ys = [[],[],[],[],[]]

    # COMPUTE BASELINE METRIC
    threshold = [1.5, 2.5, 3.5, 4.5, 5.5]
    pred2 = pd.cut(pred, [-np.inf] + threshold + [np.inf], 
                    labels=[1,2,3,4,5,6]).astype('int32')
    best = cohen_kappa_score(true, pred2, weights="quadratic")

    # FIND FIVE OPTIMAL THRESHOLDS
    for k in range(5):
        for sign in [1,-1]:
            v = threshold[k]
            threshold2 = threshold.copy()
            stop = 0
            while stop<steps:

                # TRY NEW THRESHOLD
                v += sign * 0.01
                threshold2[k] = v
                pred2 = pd.cut(pred, [-np.inf] + threshold2 + [np.inf], 
                                labels=[1,2,3,4,5,6]).astype('int32')
                metric = cohen_kappa_score(true, pred2, weights="quadratic")

                # SAVE TRIALS FOR PLOTTING
                xs[k].append(v)
                ys[k].append(metric)

                # EARLY STOPPING
                if metric<=best:
                    stop += 1
                else:
                    stop = 0
                    best = metric
                    threshold = threshold2.copy()

    # COMPUTE FINAL METRIC
    pred2 = pd.cut(pred, [-np.inf] + threshold + [np.inf], 
                    labels=[1,2,3,4,5,6]).astype('int32')
    best = cohen_kappa_score(true, pred2, weights="quadratic")   

    # RETURN RESULTS
    threshold = [np.round(t,3) for t in threshold]
    return best, threshold, xs, ys

# best, thresholds, xs, ys = find_thresholds(y_split, oof, steps=500)
# print('Best thresholds are:', thresholds )
# print('=> achieve Overall CV QWK score =', best )

# >> Output: thresholds = [1.5, 2.6, 3.5, 4.59, 5.55]

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, cohen_kappa_score
from aes2_added_fb_prize_as_features_preprocessing import *

models = []
predictions = []
f1_scores = []
kappa_scores = []

tuner_params = {
    "gpu_id" : 5,
    "learning_rate_lgb": 0.05,
    "max_depth_lgb": 8,
    "num_leaves_lgb": 10,
    "colsample_bytree_lgb": 0.3,
    "reg_alpha_lgb": 0.7,
    "reg_lambda_lgb": 0.01,
    "n_estimators_lgb": 700,
    "learning_rate_xgb": 0.1,
    "max_depth_xgb": 8,
    "num_leaves_xgb": 10,
    "colsample_bytree_xgb": 0.5,
    "reg_alpha_xgb": 0.1,
    "reg_lambda_xgb": 0.8,
    "n_estimators_xgb": 1024,
    "n_splits": 15
}

n_splits = tuner_params['n_splits']
oof = np.zeros(len(train), dtype='float32')

class Predictor:
    def __init__(self, models: list):
        self.models = models
#         self.xgb_boost_best_iter = models[1].
    def predict(self, X):
        n_models = len(self.models)
        predicted = None
        n = 0.749
        for i, model in enumerate(self.models):
            if i == 0:
                predicted = n*model.predict(X)
            else:
                # if not isinstance(X, xgb.DMatrix):
                #     X = xgb.DMatrix(X)
                predicted += (1-n)*model.predict(X)
        return predicted

skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)
xgboost_best_iters = []
light_best_iters = []
thresholds = [1.5, 2.5, 3.5, 4.5, 5.5] # init threshold


LOAD = True # re-train
if not LOAD:
    for i in range(n_splits):
        random_indices = np.random.choice(X.shape[0], size=5000, replace=False)
        random_test_X = X[random_indices, :]
        random_test_y = y_split[random_indices]
        lgb_predictor = lgb.Booster(model_file=f'kaggle/input/3models/aes-lgbm/fold_{i+1}.txt')
        # cat_model = CatBoostRegressor()
        # cat_model.load_model(f'kaggle/input/aes-catboost/fold_{i+1}.cbm')
        xgb_regressor = xgb.Booster()
        xgb_regressor.load_model(f'kaggle/input/3models/aes-xgboost/fold_{i+1}.bin')

        predictor = Predictor([lgb_predictor, xgb_regressor])
        
        models.append(predictor)
        
        # # Make predictions on the test data for this fold
        #         # Make predictions on the test data for this fold
        predictions_fold = predictor.predict(random_test_X)
        predictions_fold = predictions_fold + a
        predictions_fold = predictions_fold.clip(1, 6).round().astype(int)
        
        # # Confusion Matrix for each fold
        # cm = confusion_matrix(random_test_y, predictions_fold, labels=[x for x in range(1,7)])
        # disp = ConfusionMatrixDisplay(confusion_matrix=cm,
        #                               display_labels=[x for x in range(1,7)])
        # disp.plot()
        # plt.show()
        
        # # Calculate and store the F1 score for this fold
        f1_fold = f1_score(random_test_y, predictions_fold, average='weighted')
        f1_scores.append(f1_fold)

        # Calculate and store the Cohen's kappa score for this fold
        kappa_fold = cohen_kappa_score(random_test_y, predictions_fold, weights='quadratic')
        kappa_scores.append(kappa_fold)

        print(f'F1 score across fold: {f1_fold}')
        print(f'Cohen kappa score across fold: {kappa_fold}')
        gc.collect()
else:
    for i, (train_index, test_index) in enumerate(skf.split(X, y_split), 1):
    # Split the data into training and testing sets for this fold
        print('fold',i)
        X_train_fold, X_test_fold = X[train_index], X[test_index]
        y_train_fold, y_test_fold, y_test_fold_int = y[train_index], y[test_index], y_split[test_index]
        callbacks = [log_evaluation(period=25), early_stopping(stopping_rounds=75,first_metric_only=True)]
        light = lgb.LGBMRegressor(
                objective = qwk_obj,
                metrics = 'None',
                learning_rate = tuner_params["learning_rate_lgb"],
                max_depth = tuner_params["max_depth_lgb"],
                num_leaves = tuner_params["num_leaves_lgb"],
                colsample_bytree = tuner_params["colsample_bytree_lgb"],
                reg_alpha = tuner_params["reg_alpha_lgb"],
                reg_lambda = tuner_params["reg_lambda_lgb"],
                n_estimators = tuner_params["n_estimators_lgb"],
                random_state=42,
                extra_trees=True,
                class_weight='balanced',
                # device='gpu' if CUDA_AVAILABLE else 'cpu',
                # device_type="gpu",
                verbosity = - 1
            )

        # Fit the model on the training data for this fold  
        light.fit(
            X_train_fold,
            y_train_fold,
            eval_names=['train', 'valid'],
            eval_set=[(X_train_fold, y_train_fold), (X_test_fold, y_test_fold)],
            eval_metric=quadratic_weighted_kappa,
            callbacks=callbacks
        )
        light_best_iters.append(light.best_iteration_)
        
        
        light.booster_.save_model(f'kaggle/out/aes-lgbm/fold_{i}.txt')
        print('\nFold_{} LightGBM Model saved.\n'.format(i))
        
        
        xgb_callbacks = [
            xgb.callback.EvaluationMonitor(period=25),
            xgb.callback.EarlyStopping(75, metric_name="QWK", maximize=True, save_best=True)
        ]
        xgb_regressor = xgb.XGBRegressor(
            objective = qwk_obj,
            metrics = 'None',
            learning_rate = tuner_params["learning_rate_xgb"],
            max_depth = tuner_params["max_depth_xgb"],
            num_leaves = tuner_params["num_leaves_xgb"],
            colsample_bytree = tuner_params["colsample_bytree_xgb"],
            reg_alpha = tuner_params["reg_alpha_xgb"],
            reg_lambda = tuner_params["reg_lambda_xgb"],
            n_estimators = tuner_params["n_estimators_xgb"],
            random_state=42,
            extra_trees=True,
            class_weight='balanced',
            tree_method="hist",
            # device="gpu" if CUDA_AVAILABLE else "cpu",
            gpu_id = tuner_params["gpu_id"]
        #             device='gpu',
        #             verbosity = 1
        )
        
        xgb_callbacks = [
            xgb.callback.EvaluationMonitor(period=25),
            xgb.callback.EarlyStopping(75, metric_name="QWK", maximize=True, save_best=True)
        ]
        xgb_regressor.fit(
            X_train_fold,
            y_train_fold,
            eval_set=[(X_train_fold, y_train_fold), (X_test_fold, y_test_fold)],
            eval_metric=quadratic_weighted_kappa,
            callbacks=xgb_callbacks
        )
        xgboost_best_iters.append(xgb_regressor.get_booster().best_iteration)
        
        xgb_regressor.save_model(f'kaggle/out/aes-xgboost/fold_{i}.bin')
        print('\nFold_{} XGBoost Model saved.\n'.format(i)) 
        
        predictor = Predictor([light, xgb_regressor])
        
        models.append(predictor)
        # Make predictions on the test data for this fold
        predictions_fold = predictor.predict(X_test_fold)
        predictions_fold = predictions_fold + a
        
        
        predictions_fold = pd.cut(
            predictions_fold, [-np.inf] + thresholds + [np.inf], 
            labels=[1,2,3,4,5,6]
            ).astype('int32')
        oof[test_index] = predictions_fold
        
        predictions_fold = predictions_fold.clip(1, 6).round()
        predictions.append(predictions_fold)
        # Calculate and store the F1 score for this fold
        f1_fold = f1_score(y_test_fold_int, predictions_fold, average='weighted')
        f1_scores.append(f1_fold)

        # Calculate and store the Cohen's kappa score for this fold
        kappa_fold = cohen_kappa_score(y_test_fold_int, predictions_fold, weights='quadratic')
        kappa_scores.append(kappa_fold)
    #         predictor.booster_.save_model(f'fold_{i}.txt')
        cm = confusion_matrix(y_test_fold_int, predictions_fold, labels=[x for x in range(1,7)])

        disp = ConfusionMatrixDisplay(
            confusion_matrix=cm,
            display_labels=[x for x in range(1,7)]
        )
        disp.plot()
        plt.show()
        print(f'F1 score across fold: {f1_fold}')
        print(f'Cohen kappa score across fold: {kappa_fold}')

        gc.collect()
        #if ENABLE_DONT_WASTE_YOUR_RUN_TIME:
        #    break


In [9]:
mean_f1_score = np.mean(f1_scores)
mean_kappa_score = np.mean(kappa_scores)
# Print the mean scores
print(f'Mean F1 score across {n_splits} folds: {mean_f1_score}')
print(f'Mean Cohen kappa score across {n_splits} folds: {mean_kappa_score}')
# print(f"XGBoost mean best iters: {sum(xgboost_best_iters)/len(xgboost_best_iters)}")
# print(f"LightBoost mean best iters: {sum(light_best_iters)/len(light_best_iters)}")

Mean F1 score across 15 folds: 0.6728648365558038
Mean Cohen kappa score across 15 folds: 0.8341763402592286


In [10]:

# from aes2_added_fb_prize_as_features_preprocessing import preprocess_test, infer
import aes2_added_fb_prize_as_features_preprocessing

test_feats = aes2_added_fb_prize_as_features_preprocessing.preprocess_test()
test_feats.columns = clean_feature_names(test_feats.columns)
aes2_added_fb_prize_as_features_preprocessing.infer(test_feats, models)


tokenizers.__version__: 0.19.1
transformers.__version__: 4.41.1


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

tokenizers.__version__: 0.19.1
transformers.__version__: 4.41.1


Model: microsoft/deberta-v3-large Score: 0.4569  Scores: [0.4881251563991662, 0.45350791183634237, 0.4167653497073674, 0.4555629168241569, 0.47309887903749326, 0.4544346186864903]


0    000d118
Name: text_id, dtype: object
0    000d118
Name: text_id, dtype: object
CustomModel(
  (model): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 1024, padding_idx=0)
      (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-23): 24 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
     

...

  0%|          | 0/1 [00:01<?, ?it/s]

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/home/mcq/anaconda3/envs/torch/lib/python3.10/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/home/mcq/anaconda3/envs/torch/lib/python3.10/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/home/mcq/anaconda3/envs/torch/lib/python3.10/multiprocessing/connection.py", line 361, in _close
Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/home/mcq/anaconda3/envs/torch/lib/python3.10/multiprocessing/queues.py", line 239, in _feed
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/mcq/anaconda3/envs/torch/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    reader_close()
  File "/home/mcq/anaconda3/envs/torch/lib/python3.10/multiprocessing/connection.py", line 177, in close
    self.

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Features number:  21913
[2. 3. 4.]


,essay_id,score
0,000d118,2
1,000fe60,3
2,001ab80,4
